
# Assignment No 4b
###### *Sibt ul Hussain*
----
## Goal

Your goal in this part of assigment is to implement a Decision Tree Classifier for categorical variables.

**Note** Please note that you are allowed to use only those libraries which we have discussed in the class, i.e. numpy, scipy, pandas.

## Submission Instructions
You are required to submit the original notebook file on the Slate (with .ipynb extension), with complete set of outputs. Students failing to do so will get zero marks. 

*Please read each step carefully and understand it fully before proceeding with code writing*

## Plagiarism
Any form of plagiarism will not be tolerated and result in 0 marks.



### Decision Tree Classifier

Now in this assignment we will be implementing the Decision Classifier for both Continuous and Categorical attributes.

Decision tree can be built by using any of the following split criterias, namely:
 - Information Gain
 - Gini Index
 - CART 

However, you are required here to implement the decision tree with information gain as splitting criterion.

Remember in my code i am not looking for maximizing the information gain, instead i am looking for minimizing the split entropy. Recall,
$$Information Gain  = H(D) - H(D_Y,D_N)$$

Where,

$H(D)$ is the data set entroy and $H(D_Y,D_N)$ is split entropy. Since $H(D)$ is constant for the given dataset so maximizing the entropy is equal to minimizing the split entropy and that is what is being represented in my code outputs.

In [1]:
%pylab inline
import scipy.stats
#from collections import defaultdict
import pandas as pd
import tools as t
from math import log
#from sklearn.cross_validation import train_test_split
from collections import defaultdict, Counter
import time

Populating the interactive namespace from numpy and matplotlib


In [2]:
def getSplits(categories):
    '''
        function returns list of splits for the given list of categorical variables...
        
        Input:
        ------------
            categories: a list of unique categories...
        
        Return:
        ------------
            list of splits(tuples) for given list of categorical variables. Each pair of sublists
            defines the left and right splits, e.g. This list
            [('y', 'f'), ('s', 'g'), ('f'), ('y', 's', 'g')]
            
            defines two splits with each pair representing a different split.
        Examples:
        ------------
        splits=getSplits(['a1','a2','a3','a4']) will return 
        [('a1', 'a2', 'a4'), ('a3',), ('a2', 'a4'), 
        ('a1', 'a3'), ('a1', 'a4'), ('a3', 'a2'), ('a1', 'a3', 'a2'), 
        ('a4',), ('a3', 'a4'), ('a1', 'a2'), ('a1',), ('a3', 'a2', 'a4'), ('a2',), ('a1', 'a3', 'a4')]

            
    '''
    categories=set(categories)
    tsplits=t.get_powerset(categories,len(categories)-1)
    flist=[]
    for s in tsplits:
        if not s in flist:
            r=categories.difference(s)
            flist.append(s)
            flist.append(r)
    olist=[]
    for s in flist:
        ilist=[]
        for k in s:
            ilist.append(k)
        olist.append(tuple(ilist))
    return olist


In [3]:
splits=getSplits(['a1','a2','a3','a4'])
print splits,len(splits)

[('a1', 'a2', 'a4'), ('a3',), ('a2', 'a4'), ('a1', 'a3'), ('a1', 'a4'), ('a3', 'a2'), ('a1', 'a3', 'a2'), ('a4',), ('a3', 'a4'), ('a1', 'a2'), ('a1',), ('a3', 'a2', 'a4'), ('a2',), ('a1', 'a3', 'a4')] 14


In [4]:
class Node:
    def __init__(self,purity,klasslabel='',score=0,split=[],fidx=-1):
        '''
            purity: purity level at which to stop
            klasslabel: klasslabel of the node, (for leaf node)
            score: information gain of the newly added node
            split: splitting threshold
            fidx: feature index            
        '''
        self.lchild=None
        self.rchild=None
        self.klasslabel=klasslabel        
        self.split=split
        self.score=score
        self.fidx=fidx
        self.purity=purity
        self.ftype= 'categorical' if type(self.split) in [tuple, str, numpy.string_] else 'continuous' # feature type 

        
    def set_childs(self,lchild,rchild):
        # YOUR CODE HERE
        self.lchild=lchild
        self.rchild=rchild
        
    def isleaf(self):
        # YOUR CODE HERE
        if (self.lchild is None)&(self.rchild is None):
            return True
        return False
        
    def isless_than_eq(self, X):
        # YOUR CODE HERE
        if self.ftype=='categorical':
            if X[self.fidx] in self.split:
                return True
            return False
        else:
            if X[self.fidx] <= self.split:
                return True
            return False
        
    def get_str(self):        
        if self.isleaf():
            return 'C(class={},Purity={})'.format(self.klasslabel,self.purity)
        else:
            return 'I(Fidx={},Score={},Split={})'.format(self.fidx,self.score,self.split) 

In [5]:
class DecisionTree:
    def __init__(self, purityp, exthreshold, maxdepth=10):        
        self.purity = purityp
        self.exthreshold = exthreshold
        self.maxdepth = maxdepth
        pass
    def train(self, X, Y):
        nexamples, nfeatures = X.shape
        self.tree=self.build_tree(X,Y,0)
    def build_tree(self, X, Y,depth):
        nexamples, nfeatures = X.shape
        klasses,klasses_counts=np.unique(Y,return_counts=True);
        nexamples=nexamples*1.0
        leafLabel = ""
        purityd=0
        for k in klasses:
            temp = np.count_nonzero(Y[Y == k]) / nexamples
            if(temp > purityd):
                purityd = temp
                leafLabel = k
        if depth >= self.maxdepth or purityd >= self.purity or nexamples <= self.exthreshold:
            node=Node(purityd,klasslabel=leafLabel)
        else:
            splitS=+np.inf
            splitP=0
            onfe=0
            SDY=[]
            SDN=[]
            for fe in xrange(nfeatures):
                self.classes=np.unique(Y)
                self.nclasses=len(np.unique(Y))
                if type(X[0,fe]) is int  or float:
                    val,sco,dn,dy=self.evaluate_numerical_attribute(X[:,fe],Y)
                else:
                    if(len(np.unique(X[:,fe]))>1):
                        val,sco,dn,dy=self.evaluate_categorical_attribute(X[:,fe],Y)
                    else:
                        if nfeatures<2:
                            for k in klasses:
                                temp = np.count_nonzero(Y[Y == k]) / nexamples
                                if(temp > purityd):
                                    purityd = temp
                                    leafLabel = k
                            return Node(purityd,KlassLabel=leafLabel)
                        else:
                            sco=+np.inf
                if splitS > sco:
                    splitS=sco
                    splitP=val
                    SDN=dn
                    SDY=dy
                    onfe=fe
            if splitP==0:
                for k in klasses:
                                temp = np.count_nonzero(Y[Y == k]) / nexamples
                                if(temp > purityd):
                                    purityd = temp
                                    leafLabel = k
                node=Node(purityd,klasslabel=leafLabel)
            else:
                node=Node(purityd,score=splitS,split=splitP,fidx=onfe)
                node.lchild=self.build_tree(X[SDN,:],Y[SDN],depth+1)
                node.rchild=self.build_tree(X[SDY,:],Y[SDY],depth+1)
        return node
    def test(self, X):
        nexamples, nfeatures = X.shape
        pclasses = self.predict(X)
        return np.array(pclasses)
    def evaluate_categorical_attribute(self, feat, Y):
            categories = set(feat)
            splits = getSplits(categories) if len(categories) > 1 else tuple(categories)
            freq = scipy.stats.itemfreq(Y)
            f=np.array(feat)
            classes = np.unique(Y)
            nclasses = len(classes)
            csplit=0
            mingain=+np.inf
            for i in range(0,len(splits),2):
                probdn=0.0
                for atr in splits[i]:
                    probdn+=(f==atr).sum()
                probdy=len(f)-probdn
                hdn=0
                hdy=0
                for cl in classes:
                    tmp=f[Y==cl]
                    tmp1=0.0
                    for atr in splits[i]:
                        tmp1+=(tmp==atr).sum()
                    tmp1=tmp1*1.0
                    if tmp1>0:
                        hdn+=(tmp1/probdn)*np.log2(tmp1/probdn)
                    tmp=f[Y!=cl]
                    tmp1=0
                    for atr in splits[i+1]:
                        tmp1+=(tmp==atr).sum()
                    tmp1=tmp1*1.0
                    if tmp1>0:
                        hdy+=(tmp1/probdy)*np.log2(tmp1/probdy)
                hdn=hdn*((probdn*1.0)/(probdn+probdy))*(-1)
                hdy=hdy*((probdy*1.0)/(probdn+probdy))*(-1)
                if ((hdn+hdy) < mingain):
                    csplit=splits[i]
                    mingain=hdn+hdy
            Xlidx=(f==csplit[0])
            for i in range(1,len(csplit)):
                Xlidx=np.asarray(f==csplit[i])|np.asarray(Xlidx)
            Xridx=~Xlidx
            return csplit, mingain, Xlidx, Xridx
    def evaluate_numerical_attribute(self, feat, Y):
        classes = np.unique(Y)
        nclasses = len(classes)
        sidx=np.argsort(feat)
        f=feat[sidx]
        sY=Y[sidx]
        Midpoints=set()
        nt=defaultdict(int)
        N={}
        for i in xrange(nclasses):
            nt[classes[i]]=0
        for i in xrange(len(f)-1):
            nt[sY[i]]+=1
            if f[i]!=f[i+1]:
                v=(f[i]+f[i+1])/2
                Midpoints.add(v)
                N[v]=nt.values()
        nt[sY[len(f)-1]]+=1
        n={}
        for i,tmp in enumerate(nt.keys()):
            n[i]=nt[tmp]*1.0
        split=0
        mingain=+np.inf
        for v in Midpoints:
            en_dy=0.0
            en_dn=0.0
            for i in xrange(nclasses):
                tmp=N[v][i]*1.0
                if tmp > 0.0:
                    en_dy+=(tmp/sum(N[v]))*np.log2(tmp/sum(N[v]))
                tmp1=(n[i]-tmp)*1.0
                if tmp1 > 0.0:
                    en_dn+=(tmp1/sum(n.values()-np.array(N[v])))*np.log2(tmp1/sum(n.values()-np.array(N[v])))
            tmp=((sum(N[v])*1.0)/sum(n.values()))
            gain=(-1)*((en_dy*tmp)+(en_dn*(1.0-tmp)))
            if (gain < mingain):
                split=v
                mingain=gain
        temp=np.array((feat))
        Xlidx=temp<split
        Xridx=temp>split
        return split,mingain,Xlidx,Xridx
    def predict(self, X):
        z = []
        for idx in range(X.shape[0]):
            z.append(self._predict(self.tree, X[idx, :]))
        return z 
    def _predict(self, node, X):
        if node.isleaf():
            return node.klasslabel
        if node.isless_than_eq(X):
            return self._predict(node.lchild,X)
        return self._predict(node.rchild,X)
    def __str__(self):
        str = '---------------------------------------------------'
        str += '\n A Decision Tree With Depth={}'.format(self.find_depth())
        str += self.__print(self.tree)
        str += '\n---------------------------------------------------'
        return str
    def find_depth(self):
        return self._find_depth(self.tree)
    def _find_depth(self, node):
        if not node:
            return
        if node.isleaf():
            return 1
        else:
            return max(self._find_depth(node.lchild), self._find_depth(node.rchild)) + 1
    def __print(self, node, depth=0):
        ret = ""
        if node.rchild:
            ret += self.__print(node.rchild, depth + 1)
        ret += "\n" + ("    "*depth) + node.get_str()
        if node.lchild:
            ret += self.__print(node.lchild, depth + 1)
        return ret

# Lets test our code for the given example in the book.

In [6]:
#load the Iris dataset
tdata=pd.read_csv('./iris.data', header=None)
tdata.columns=['SepalLength','SepalWidth','PetalLength','PetalWidth','Class']

In [7]:
tx=tdata['SepalLength'].dropna()
tx[(tdata['SepalLength']>=4.3) & (tdata['SepalLength']<=5.2)]='a1'
tx[(tdata['SepalLength']>5.2) & (tdata['SepalLength']<=6.1)]='a2'
tx[(tdata['SepalLength']>6.1) & (tdata['SepalLength']<=7.0)]='a3'
tx[(tdata['SepalLength']>7.0) & (tdata['SepalLength']<=7.9)]='a4'

In [8]:
print tx.values
Y=tdata['Class'].dropna()
Y[Y=='Iris-virginica']='Iris-versicolor'
Y=Y.values
print Y

['a1' 'a1' 'a1' 'a1' 'a1' 'a2' 'a1' 'a1' 'a1' 'a1' 'a2' 'a1' 'a1' 'a1' 'a2'
 'a2' 'a2' 'a1' 'a2' 'a1' 'a2' 'a1' 'a1' 'a1' 'a1' 'a1' 'a1' 'a1' 'a1' 'a1'
 'a1' 'a2' 'a1' 'a2' 'a1' 'a1' 'a2' 'a1' 'a1' 'a1' 'a1' 'a1' 'a1' 'a1' 'a1'
 'a1' 'a1' 'a1' 'a2' 'a1' 'a3' 'a3' 'a3' 'a2' 'a3' 'a2' 'a3' 'a1' 'a3' 'a1'
 'a1' 'a2' 'a2' 'a2' 'a2' 'a3' 'a2' 'a2' 'a3' 'a2' 'a2' 'a2' 'a3' 'a2' 'a3'
 'a3' 'a3' 'a3' 'a2' 'a2' 'a2' 'a2' 'a2' 'a2' 'a2' 'a2' 'a3' 'a3' 'a2' 'a2'
 'a2' 'a2' 'a2' 'a1' 'a2' 'a2' 'a2' 'a3' 'a1' 'a2' 'a3' 'a2' 'a4' 'a3' 'a3'
 'a4' 'a1' 'a4' 'a3' 'a4' 'a3' 'a3' 'a3' 'a2' 'a2' 'a3' 'a3' 'a4' 'a4' 'a2'
 'a3' 'a2' 'a4' 'a3' 'a3' 'a4' 'a3' 'a2' 'a3' 'a4' 'a4' 'a4' 'a3' 'a3' 'a2'
 'a4' 'a3' 'a3' 'a2' 'a3' 'a3' 'a3' 'a2' 'a3' 'a3' 'a3' 'a3' 'a3' 'a3' 'a2']
['Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setos

In [9]:
%pdb off
dt=DecisionTree(0.95,5,5)
split, gain, Xlidx, Xridx = dt.evaluate_categorical_attribute(tx,Y)
print 
print split, gain

Automatic pdb calling has been turned OFF

('a1',) 0.508690715247


In [10]:
from nose.tools import assert_almost_equal, assert_almost_equals

%pdb off
dt=DecisionTree(0.95,5,5)
split, gain, Xlidx, Xridx = dt.evaluate_categorical_attribute(tx,Y)


assert_almost_equal('a1', split[0])
assert_almost_equal(gain, 0.51, 1)

Automatic pdb calling has been turned OFF


In [19]:
from nose.tools import assert_greater_equal
Xtrain,Ytrain,Xtest,Ytest=t.split_data(X,Y)
feat=np.arange(2)
dt=DecisionTree(0.95,5,10)
dt.train(Xtrain[:,feat],Ytrain)


pclasses=dt.predict(Xtest[:,feat])
acc = np.sum(pclasses==Ytest)/float(Ytest.shape[0])

assert_greater_equal(acc, 0.60)

In [20]:
from nose.tools import assert_greater_equal
Xtrain,Ytrain,Xtest,Ytest=t.split_data(X,Y)

dt=DecisionTree(0.95,5,10)
dt.train(Xtrain,Ytrain)

pclasses=dt.predict(Xtest)
acc = np.sum(pclasses==Ytest)/float(Ytest.shape[0])

assert_greater_equal(acc, 0.90)